# Get images from Adobe repo

- take random row

In [1]:
import csv

import glob
import pandas as pd
import sys
# Those CSV-files are too big and we need these two lines
csv.field_size_limit(sys.maxsize)
import random

TOPK = 5
IMAGE_ID = 0
USER_ID = 1
USER_TAGS = 8
MACHINE_TAGS = 9
IMAGE_URL = 14
PHOTO_OR_VIDEO = 22
SUBSET = 100

# rows = []
images = []
tag2image = {}
image_tags = []
for i in glob.glob('/mnt/ilcompf2d1/data/yfcc100m/yfcc100m_dataset-0*'):
    with open(i, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quoting=csv.QUOTE_NONE)
        for j, row in enumerate(reader):
            if int(row[PHOTO_OR_VIDEO]) != 0 or len(row[USER_TAGS]) == 0:
                continue
            if SUBSET is not None:
                if random.random() > SUBSET / 10000000:
                    continue 
            break

data at adobe is: `/mnt/ilcompf2d1/data/yfcc100m/`

- find the image in the adobe data

- show image

In [19]:
!wc -l $i

10000000 /mnt/ilcompf2d1/data/yfcc100m/yfcc100m_dataset-0


In [2]:
print(j)
print(row)

494130
['3948397176', '33938182@N04', 'Lucy+Takakura', '2009-09-21 13:11:55.0', '1253727701', 'Panasonic+DMC-TZ1', '%E3%82%B8%E3%83%A7%E3%83%B3%E3%83%98%E3%83%B3%E3%83%AA%E3%83%BC%E3%82%BA%E3%82%B9%E3%82%BF%E3%83%87%E3%82%A3', 'shot+by+Lumix+DMC-TZ1', 'lumixdmc-tz1', '', '', '', '', 'http://www.flickr.com/photos/33938182@N04/3948397176/', 'http://farm3.staticflickr.com/2643/3948397176_22a2a41211.jpg', 'Attribution-ShareAlike License', 'http://creativecommons.org/licenses/by-sa/2.0/', '2643', '3', '22a2a41211', '2d609ea749', 'jpg', '0']


In [23]:
# (10000000 - 1) // 10000
494130 // 10000

49

In [24]:
494130 % 10000

4130

In [25]:
filename = '/mnt/ilcompf2d1/data/yfcc100m/image/0-049.tar'
import tarfile

reader = tarfile.open(filename)
fid = reader.extractfile('0-049/0494130.jpg')
assert fid is None

In [30]:
from PIL import Image

img = Image.open(fid)
# dump image
# img.save('hola.jpg')

Going back to row index

In [3]:
49 * 10000 + 4130

494130

# YFCC100M $\cap$ DiDeMo

__Note__: You can also find the following cell into `scripts/` due to the long time they take too executed (> 30mins).

## 1. Get subset

In [30]:
import json
import numpy as np

def create_vocabulary(underrepresented_threshold=10):
    "Create vocabulary of underrepresented NOUNs to augment"

    # load stats about nouns in DIDEMO
    NOUNS_FILE = '../data/interim/didemo/nouns_to_video.json'
    with open(NOUNS_FILE, 'r') as fid:
        stats = json.load(fid)
        for k, v in stats['nouns_per_subset'].items():
            stats['nouns_per_subset'][k] = set(v)
    
    # load annotations in val-set
    VAL_FILE = '../data/raw/val_data.json'
    with open(VAL_FILE, 'r') as fid:
        data = json.load(fid)
    num_val_instances = len(data)
    spanned_annotations = []
    for k, v in stats['annotations_per_subset']['val'].items():
        spanned_annotations.extend(v)
    spanned_annotations = np.unique(spanned_annotations)
    
    nouns_vocab = set()
    # TODO add val_ids to prioritize verification step
    nouns_and_metadata = {}
    num_nouns_train = len(stats['counts_per_subset']['train'])
    num_nouns_underrepresented = 0
    val_ids_toimpact = []
    val_ids_represented = []
    for k, v in stats['counts_per_subset']['train'].items():
        if v <= underrepresented_threshold:
            num_nouns_underrepresented += 1
            if k in stats['counts_per_subset']['val']:
                val_ids_toimpact.extend(
                    stats['annotations_per_subset']['val'][k])
                nouns_vocab.add(k)
        else:
            if k in stats['counts_per_subset']['val']:
                val_ids_represented.extend(
                    stats['annotations_per_subset']['val'][k])

    num_samples_well_represented = len(np.unique(val_ids_represented))
    nouns_only_val = (stats['nouns_per_subset']['val'] -
                      stats['nouns_per_subset']['train'])
    num_nouns_notrepresented = len(nouns_only_val)
    for k in nouns_only_val:
        val_ids_toimpact.extend(
            stats['annotations_per_subset']['val'][k])
        nouns_vocab.add(k)
    num_samples_to_impact = len(np.unique(val_ids_toimpact))

    print('Num evaluation instances:', num_val_instances)
    print('Spanned instances', len(spanned_annotations))
    print('NOUNs are underrepresented when appear less than', underrepresented_threshold + 1)
    print('Total NOUNs in train', num_nouns_train)
    print('NOUNs Under-represented', num_nouns_underrepresented)
    print('NOUNs Unseen during training', num_nouns_notrepresented)
    print('Num descriptions with Under&Unseen NOUNs', num_samples_to_impact)
    print('Num descriptions with Well NOUNs', num_samples_well_represented)
    print('Pctg to impact', f'{num_samples_to_impact / num_val_instances:.4f}')
    return nouns_vocab

nouns = create_vocabulary(75)

Num evaluation instances: 4180
Spanned instances 4156
NOUNs are underrepresented when appear less than 76
Total NOUNs in train 3521
NOUNs Under-represented 3358
NOUNs Unseen during training 167
Num descriptions with Under&Unseen NOUNs 2119
Num descriptions with Well NOUNs 3814
Pctg to impact 0.5069


In [ ]:
TOPK = 5
IMAGE_ID = 0
USER_ID = 1
USER_TAGS = 8
MACHINE_TAGS = 9
IMAGE_URL = 14
PHOTO_OR_VIDEO = 22
IMAGES_PER_TAR = 10000
TRAINABLE_THRESHOLD = 1000
UNDERREPRESENTED_THRESHOLD = 75

# Load clean (scrapped) tags
import csv
filename = '../data/interim/yfcc100m/tag_frequency.csv'
clean_tags = set()
with open(filename, 'r') as fid:
    reader = csv.DictReader(fid, delimiter=',')
    for row in reader:
        clean_tags.add(row['tag'])

# load a set of interesting NOUNs from DiDeMo
didemo_nouns = create_vocabulary(UNDERREPRESENTED_THRESHOLD)

# Lemmatizer to deal with plurals
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

# Extract mapping tags 2 image
import glob
import pandas as pd
import sys
# Those CSV-files are too big and we need these two lines
csv.field_size_limit(sys.maxsize)
tag2image = {}
images = []
image_tags_topk_flickr = []
image_tags_flickr = []
image_tags_topk = []
image_tags = []
image_urls = []
for filename in glob.glob('/mnt/ilcompf2d1/data/yfcc100m/yfcc100m_dataset-*'):
    with open(filename, newline='') as csvfile:
        file_index = filename.split('-')[1]
        reader = csv.reader(csvfile, delimiter='\t', quoting=csv.QUOTE_NONE)
        for line_no, row in enumerate(reader):
            if int(row[PHOTO_OR_VIDEO]) != 0 or len(row[USER_TAGS]) == 0:
                continue

            counter, amen = 0, False
            tags_topk_flickr = []
            tags_topk = []
            for tag in row[USER_TAGS].split(','):
                lemmatized_tag = lemmatizer(tag, u'NOUN')[0]
                if lemmatized_tag not in clean_tags:
                    continue
                if lemmatized_tag not in didemo_nouns:
                    continue
                # after this point the image will be considered, thus we
                # can add it to tag2image dict
                counter += 1
                tags_topk.append(lemmatized_tag)
                tags_topk_flickr.append(tag)
                amen = True
                if lemmatized_tag in tag2image:
                    tag2image[lemmatized_tag].append(len(images))
                else:
                    tag2image[lemmatized_tag] = [len(images)]
                if counter == TOPK:
                    # according to previous Bryan's project top-5 tags in YFCC100M
                    # have purity >= 60%
                    break
            tags = [i for i in row[USER_TAGS].split(',')
                    if lemmatizer(i, u'NOUN')[0] in clean_tags]

            # I don't need this but just in case
            if not amen:
                continue
            tar_index = line_no // IMAGES_PER_TAR
            img_index = line_no % IMAGES_PER_TAR
            image_loc = f'{file_index}-{tar_index:03d}/{tar_index:03d}{img_index:04d}.jpg'
            image_url = row[IMAGE_URL]
            images.append(image_loc)
            image_urls.append(image_url)
            image_tags_topk_flickr.append(';'.join(tags_topk_flickr))
            image_tags_flickr.append(row[USER_TAGS].replace(',', ';'))
            image_tags_topk.append(';'.join(tags_topk))
            image_tags.append(';'.join(tags))

print('Num clean tags:', len(clean_tags))

import json
import numpy as np
NOUNS_FILE = '../data/interim/didemo/nouns_to_video.json'
with open(NOUNS_FILE, 'r') as fid:
    didemo_stats = json.load(fid)

trainable_tags = 0
val_instances = []
for k, v in tag2image.items():
    if len(v) > TRAINABLE_THRESHOLD:
        trainable_tags += 1
    val_instances.extend(didemo_stats[k])
val_instances = np.unique(val_instances)
print(f'Top-{TOPK} gives {len(tag2image)} '
      f'and {trainable_tags} >= {TRAINABLE_THRESHOLD} images')
print(f'Number of val instances with those NOUNs {len(val_instances)}')

import pandas as pd
df = pd.DataFrame([images, image_urls,
                   image_tags_topk, image_tags,
                   image_tags_topk_flickr, image_tags_flickr]).T
df.columns = ['adobe_cil', 'url', 'topk_tags', 'tags', 'topk_tags_yfcc100m', 'tags_yfcc100m']
basename = (f'yfcc100m_images_intersect_didemo_under-and-not-nouns-leq'
            f'-{UNDERREPRESENTED_THRESHOLD}_topk-{TOPK}')
df.to_csv(f'{basename}.csv')
with open(f'{basename}.json', 'w') as fid:
    json.dump(tag2image, fid)

## 2. Unpack and sub-sample of YFCC100M relevant for DiDeMo training

In [ ]:
import os
import random
import tarfile
from pathlib import Path
import pandas as pd
from PIL import Image

FILENAME = Path('../scripts/yfcc100m_images_intersect_didemo_under-and-not'
                '-nouns-leq-150_topk-1.csv')
OUTPUT_DIR = Path('/mnt/ssd/tmp/yfcc100m')
ROOT_DIR = Path('/mnt/ilcompf2d1/data/yfcc100m/image')
SAMPLES_PER_TAG = 1000
SEED = 1701
OUTPUT_FILE = Path(f'../data/interim/yfcc100m/sample-{SEED}_intersect-didemo'
                   '_under-and-not-nouns-leq-150_topk-1')


random.seed(SEED)
df = pd.read_csv(FILENAME, index_col=0)
df['adobe_cil_entry'] = df['adobe_cil'].apply(os.path.dirname)

# subsample per NOUNs
df_gbl = df.groupby('topk_tags')
ind = []
for label, df_i in df_gbl:
    ind_i = df_i.index.tolist()
    random.shuffle(ind_i)
    ind.extend(ind_i[:min(len(ind_i), SAMPLES_PER_TAG)])

if not OUTPUT_DIR.exists():
    os.makedirs(OUTPUT_DIR)
df = df.loc[ind, :]
total_images = len(df)
progress = 0
skipped = 0
df_gbep = df.groupby('adobe_cil_entry')
for entry, grouped in df_gbep:
    tar_file = ROOT_DIR / (entry + '.tar')
    reader = tarfile.open(tar_file)
    members = set(reader.getnames())
    for i, row in grouped.iterrows():
        if not row['adobe_cil'] in members:
            progress += 1
            skipped += 1
            continue

        fid = reader.extractfile(row['adobe_cil'])
        assert fid is not None
        img = Image.open(fid).convert('RGB')
        img_file = OUTPUT_DIR / row['adobe_cil'].replace('/', '_')
        img.save(img_file)

        progress += 1
        if progress % (total_images // 10) == 0:
            print(f'[{progress}/{total_images}]')
df.to_csv(OUTPUT_FILE, index=None)

Count instances found in YFCC100M

In [1]:
import json
import numpy as np

filename = '../scripts/yfcc100m_images_intersect_didemo_under-and-not-nouns-leq-75_topk-1.json'
with open(filename, 'r') as fid:
    data = json.load(fid)
print('Num labels to augment', len(data))

val_instances = []
for noun in data:
    val_instances.extend(stats['annotations_per_subset']['val'][noun])
val_instances = np.unique(val_instances)
print('Num instances didemo', len(val_instances))

Num labels to augment 388
Num instances didemo 1137


Stats regarding number of images

In [6]:
import numpy as np

blah = []
for k, v in tag2image.items():
    if len(v) > TRAINABLE_THRESHOLD:
        blah.append(len(v))

print(f'{TOPK},'
      f'{len(images)},'
      f'{trainable_tags},'
      f'{np.median(blah)},'
      f'{np.max(blah)},'
      f'{np.min(blah)},'
      f'{np.mean(blah)},'
      f'{np.std(blah)}')

1,1596205,25,27525.0,682898,4976,63848.2,129706.97083194874


top_k,num_images,trainable_tags,median_per_tag,max_per_tag,min_per_tag,mean_per_tag,std_per_tag

1,30232885,820,13102.0,871782,1011,36866.656097560975,79653.0784979082

3,30232934,823,25048.0,991906,1115,62644.690157958685,117116.83888841543

5,30232221,822,28745.5,1118493,1044,69639.30535279805,126868.87990805514

In [1]:
num_images = 432749
feat_dim = 2048
batch_size = 256
MAGIC_TIME = 2
print(num_images * 4 * 2048 / 1024 / 1024/ 1024)
print(MAGIC_TIME * num_images / 256 / 3600 )

3.3016128540039062
0.9391254340277778


Worst case we will spend ~XGB of space in features. We could shard blocks of images and hopefully see some compression.

We might extract all those features in around ~Xhours in a single GPU. Disregarding transfer and unpacking from huge repository.

# Images for features extraction

In [10]:
import pandas as pd
import os
from pathlib import Path

FILENAME = '../data/interim/yfcc100m/intersect_didemo/sample-1701_under-and-not-nouns-leq-150_topk-1.csv'
OUTPUT_FILE = '../data/interim/yfcc100m/intersect_didemo/feature-extraction_sample-1701_under-and-not-nouns-leq-150_topk-1.csv'
DIRNAME = Path('/mnt/ssd/tmp/yfcc100m/')
df = pd.read_csv(FILENAME)
ind = df['adobe_cil'].apply(lambda x: os.path.exists(DIRNAME / x.replace('/', '_')))
df['image'] = df['adobe_cil'].apply(lambda x: x.replace('/', '_'))
df_ = df.loc[ind, ['image']]
df_.to_csv(OUTPUT_FILE, index=None)

## T-SNE experiment

Metadata for experiment

In [2]:
import h5py
import numpy as np
import pandas as pd
import json
import os
import random
from pathlib import Path

OUTPUT_FILE = '../data/interim/yfcc100m/intersect_didemo/tsne-metadata_under-and-not-nouns-leq-150_topk-1.tsv'
TAGS = [
    'penguin',
    'mural',
    'canoe',
    'tattoo',
    'parrot',
    'playground',
    'crab',
    'scuba',
    'cow',
    'bride',
    'tram',
    'owl',
    'fog'
]
keys = []
times = []
ind_label = []

# Forming video meta-data
NOUNS_FILE = '../data/interim/didemo/nouns_to_video.json'
with open(NOUNS_FILE, 'r') as fid:
    data = json.load(fid)
    for i, tag in enumerate(TAGS):
        video_names_i = data['videos'][tag]
        keys.extend(video_names_i)
        ind_label.extend([i] * len(video_names_i))
        times.extend(data['time'][tag])
num_videos = len(keys)

# Forming image meta-data
IMG_FILE = '../data/interim/yfcc100m/intersect_didemo/sample-1701_under-and-not-nouns-leq-150_topk-1.csv'
HDF5_FILE = '/mnt/ssd/tmp/didemo/resnet152.h5'
SAMPLES_PER_TAG = 100
SEED = 1701

fid = h5py.File(HDF5_FILE, 'r')
df = pd.read_csv(IMG_FILE)
df['key'] = df['adobe_cil'].apply(lambda x: x.replace('/', '_'))
df_gbt = df.groupby('topk_tags')
for i, tag in enumerate(TAGS):
    df_i = df_gbt.get_group(tag)
    ind = [i for i, row in df_i.iterrows()
           if row['key'] in fid]
    random.shuffle(ind)
    assert len(ind) >= SAMPLES_PER_TAG
    ind = ind[:SAMPLES_PER_TAG]
    ind_label.extend([i] * SAMPLES_PER_TAG)
    times.extend([None] * SAMPLES_PER_TAG)
    keys.extend(df.loc[ind, 'key'].tolist())
fid.close()
    
ind_source = np.concatenate([[0] * num_videos,
                             [1] * (len(keys) - num_videos)])
tags = np.array(TAGS)
source = np.array(['video', 'image'])
df = pd.DataFrame({'key': keys,
                   'time': times,
                   'label': tags[ind_label],
                   'source': source[ind_source]})
df['source_label'] = df['source'].astype('str') + '_' + df['label']
df.to_csv(OUTPUT_FILE, index=None, sep='\t')

### script for T-SNE visualization

After (wasting) time, setting up everything in the cluster I found a website where we only need to pass the data 🤬

[website](https://projector.tensorflow.org/)

BTW, it seems that all the computation is done in from the browser in your laptop. Thus, it does not matter if you use the ommented part or the web.

In [ ]:
import ast
import os
import h5py
import numpy as np
import pandas as pd
#import tensorflow as tf
#from tensorflow.contrib.tensorboard.plugins import projector

FILENAMES = ['../data/interim/didemo/resnet152/320x240_max.h5',
             '../data/interim/yfcc100m/resnet152/320x240_001.h5']
METADATA = '../data/interim/yfcc100m/intersect_didemo/tsne-metadata_under-and-not-nouns-leq-150_topk-1.tsv'
OUTPUTDATA = '../data/interim/yfcc100m/intersect_didemo/tsne-data_under-and-not-nouns-leq-150_topk-1.tsv'
LOG_DIR = 'scripts/log'
D = 2048

class HDF5sOnlyDatasets():
    def __init__(self, filenames):
        self.files = filenames

    def __getitem__(self, key):
        name, file_ind = key
        with h5py.File(self.files[file_ind], 'r') as fid:
            return fid[name][:]

repo = HDF5sOnlyDatasets(FILENAMES)
df = pd.read_csv(METADATA, sep='\t')
df['time'] = df['time'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else None)
data = np.zeros((len(df), D), dtype=np.float32)
for i, row in df.iterrows():
    src_name = row['key']
    if row['time']:
        start, end = row['time']
        feat = repo[src_name, 0]
        data[i, :] = np.mean(feat[start:end + 1, :], axis=0)
    else:
        data[i, :] = repo[src_name, 1][0, :]

if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

data = pd.DataFrame(data)
data.to_csv('data_all.tsv', sep='\t', index=None, header=False)

# This work but it runs locally so I found more practical to use the projector (if u have internet XD)
# embedding_vars = []
# metadata = []
# # TODO: add dimension
# embedding_vars.append(tf.get_variable("all", initializer=data))
# for tag, df_i in df.groupby('label'):
#     ind = df_i.index
#     embedding_vars.append(tf.get_variable(tag, initializer=data[ind, :]))
# embedding_var = embedding_vars[0]

# summary_writer = tf.summary.FileWriter(LOG_DIR)

# # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
# config = projector.ProjectorConfig()
# # You can add multiple embeddings. Here we add only one.
# embedding = config.embeddings.add()
# embedding.tensor_name = embedding_var.name
# # Link this tensor to its metadata file (e.g. labels).
# embedding.metadata_path = os.path.join(os.getcwd(), METADATA)
# # The next line writes a projector_config.pbtxt in the LOG_DIR.
# # TensorBoard will read this file during startup.
# projector.visualize_embeddings(summary_writer, config)

# # Specify where you find the sprite (we will create this later)
# # embedding.sprite.image_path = path_for_mnist_sprites #'mnistdigits.png'
# # embedding.sprite.single_image_dim.extend([28,28])

# sess = tf.InteractiveSession()
# sess.run(tf.global_variables_initializer())
# saver = tf.train.Saver()
# saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 1)

# Create train/val partition from YFCC100M

Last update: July 26. created: July 24.

__Note__: Abandon this as we just wanna focus on improving video embedding.

I only tackle the minimum minimorum to make sure that partitions are disjoint, filter by uploader.

We lost uploader ID while sampling dataset. Thus, we need to trace it back.

- We can do that base on the 'adobe_cil' field. In the following way:

    - Get the first digit from the basename of dirname.
    
    - 49 * 10000 + 4130

In [ ]:
import pandas as pd
import os

IMAGE_FILE = '../data/interim/yfcc100m/intersect_didemo/sample-1701_under-and-not-nouns-leq-150_topk-1.csv'
CSV_YFCC100M_FMT = '/mnt/ilcompf2d1/data/yfcc100m/yfcc100m_dataset-{}'
IMAGES_PER_TAR = 10000
IMAGE_ID = 0
USER_ID = 1

def get_back_to_row(filename):
    filename = os.path.basename(os.path.splitext(filename)[0])
    upper = int(filename[:3])
    lower = int(filename[3:])
    return upper * 10000 + lower
    
df = pd.read_csv(IMAGE_FILE)
df['adobe_csv_index'] = df['adobe_cil_entry'].apply(lambda x: x[0])
df['row_index'] = df['adobe_cil'].apply(get_back_to_row)
df['image_id'] = 'dummy'
df['user_id'] = 'dummy'
df['row_index'] = df['adobe_cil'].apply(get_back_to_row)
for index, df_i in df.groupby('adobe_csv_index'):
    df_origin = pd.read_csv(CSV_YFCC100M_FMT.format(index), header=None, sep='\t')
    ind = df_i.index
    df_i.loc[ind, 'user_id'] = df_origin.loc[df_i.loc[:, 'row_index'], USER_ID].values
    df_i.loc[ind, 'image_id'] = df_origin.loc[df_i.loc[:, 'row_index'], IMAGE_ID].values

# Re-dump CSV
columns = set(df.columns.tolist()) - {'adobe_csv_index', 'row_index'}
df.to_csv(IMAGE_FILE, index=None, columns=columns)

## Cleaning CSV with dumped features

Some images were not in the repo, those we gotta discard them. This cell cleans the CSV such that we only pass valid images to the next stage.

In [1]:
import h5py
import pandas as pd

IMAGE_FILE = '../data/interim/yfcc100m/intersect_didemo/sample-1701_under-and-not-nouns-leq-150_topk-1.csv'
OUTPUT_FILE = '../data/interim/yfcc100m/001.csv'
HDF5_FILE = '../data/interim/yfcc100m/resnet152/320x240_001.h5'
COLUMNS_OF_INTEREST = ['key', 'topk_tags']

df = pd.read_csv(IMAGE_FILE)
df['h5_id'] = df['adobe_cil'].apply(lambda x: x.replace('/', '_'))
with h5py.File(HDF5_FILE, 'r') as fid:
    ind = df['h5_id'].apply(lambda x: x in fid)
df = df.loc[ind, :]
df.to_csv(OUTPUT_FILE, index=None)

# Create new JSON for training

created: July 26. Last update: July 24.

- We add a fake 'time' to image [0, 0] to index the feature matrix properly.

- We add a new `bool` field to all instances called `source`. It's integer representing the source of the data.

In [2]:
import pandas as pd
import json

DIDEMO_FILE = '../data/raw/train_data.json'
IMG_FILE = '../data/interim/yfcc100m/001.csv'
TRAIN_FILE = '../data/interim/didemo_yfcc100m/train_data.json'

# Unused because JSON cannot serialize number > int64
def get_max_annotation_id():
    max_annotation_id = -1000
    for i in ['train', 'val', 'test']:
        with open(f'../data/raw/{i}_data.json', 'r') as fid:
            i_instances = json.load(fid)
        df = pd.DataFrame(i_instances)
        max_annotation_id = max(max_annotation_id, df['annotation_id'].max())
    return max_annotation_id

with open(DIDEMO_FILE, 'r') as fid:
    didemo_instances = json.load(fid)
for instance in didemo_instances:
    instance['source'] = 0
    
assert didemo_instances[0]['source'] == 0
offset_annotation_id = get_max_annotation_id() + 1

df = pd.read_csv(IMG_FILE)
for i, row in df.iterrows():
    instance_i = {'video': row['h5_id'],
                  'times': [[0, 0]],
                  'description': row['topk_tags'],
                  'num_segments': None,
                  'source': 1,
                  'annotation_id': None}
                  # 'annotation_id': i + offset_annotation_id}
    didemo_instances.append(instance_i)

with open(TRAIN_FILE, 'w') as fid:
    json.dump(didemo_instances, fid)
# TODO: NOT HARD-CODE THIS
with open(TRAIN_FILE.replace('data.json', 'metadata.csv'), 'w') as fid:
    fid.write('source,id\n')
    fid.write('didemo/train_data.json,0\n')
    fid.write('yfcc100m/001.csv,1\n')

Dump list for training

In [2]:
import pandas as pd
import json

IMG_FILE = '../data/interim/yfcc100m/001.csv'
TRAIN_FILE = '../data/interim/yfcc100m/train_02/train_data.json'

didemo_instances = []
df = pd.read_csv(IMG_FILE)
for i, row in df.iterrows():
    instance_i = {'video': row['h5_id'],
                  'times': [[0, 0]],
                  'description': row['tags'].split(';'),
                  'num_segments': None,
                  'source': 1,
                  'annotation_id': None}
                  # 'annotation_id': i + offset_annotation_id}
    didemo_instances.append(instance_i)

!mkdir -p $(dirname $TRAIN_FILE)
with open(TRAIN_FILE, 'x') as fid:
    json.dump(didemo_instances, fid)
# Please edit this to keep record of what u do
with open(TRAIN_FILE.replace('train_data.json', 'metadata.csv'), 'x') as fid:
    fid.write('source,id\n')
    fid.write('yfcc100m/001.csv,1\n')

# Merging DiDeMo and YFCC100M tags

In [6]:
import pandas as pd

tags, rows = {}, []
found_tags = 0
file_ref = 'data/interim/didemo/nouns_count.csv'
filename = 'data/interim/yfcc100m/tag_frequency.csv'
newfile = 'data/interim/didemo/nouns_yfcc100m.csv'

# Get didemo tags
with open(file_ref) as fid:
    i = 0
    for line in fid:
        if i == 0:
            i += 1
            continue
        tag, count = line.strip().split(',')
        tags[tag] = None
        rows.append({'tag': tag, 'instances': count, 'dataset': 'DiDeMo'})

# Add YFCC100M tags that are in Didemo
with open(filename) as fid:
    i = 0
    for line in fid:
        if i == 0:
            i += 1
            continue
        tag, count = line.strip().split(',')
        if tag in tags:
            found_tags += 1
            rows.append({'tag': tag, 'instances': count, 'dataset': 'YFCC100M'})
print(f'Tags found: {found_tags}')

df = pd.DataFrame(rows)
#df.to_csv(newfile, index=None)

Tags found: 951


{'url': 'altair-data-41217a5eb806a0e8ca497668ef5f1003.csv',
 'format': {'type': 'csv'}}

- Take NOUNs that only appear in validation

# Map tags to images

In [2]:
%%time
# Load clean (scrapped) tags
import csv
filename = '../data/interim/yfcc100m/tag_frequency.csv'
clean_tags = set()
with open(filename, 'r') as fid:
    reader = csv.DictReader(fid, delimiter=',')
    for row in reader:
        clean_tags.add(row['tag'])

# Lemmatizer to deal with plurals
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

# Extract mapping tags 2 image
import glob
import pandas as pd
import sys
# Those CSV-files are too big and we need these two lines
csv.field_size_limit(sys.maxsize)

TOPK = 5
IMAGE_ID = 0
USER_ID = 1
USER_TAGS = 8
MACHINE_TAGS = 9
IMAGE_URL = 14
PHOTO_OR_VIDEO = 22

# rows = []
images = []
tag2image = {}
image_tags = []
for i in glob.glob('/mnt/ilcompf2d1/data/yfcc100m/yfcc100m_dataset-0*'):
    with open(i, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            if int(row[PHOTO_OR_VIDEO]) != 0 or len(row[USER_TAGS]) == 0:
                continue 
            
            counter = 0
            image_url = f'{row[IMAGE_URL]}'
            images.append(image_url)
            image_tags.append([])
            for tag in row[USER_TAGS].split(','):                    
                lemmatized_tag = lemmatizer(tag, u'NOUN')[0]
                if lemmatized_tag not in clean_tags:
                    continue
                
                counter += 1
                if lemmatized_tag in tag2image:
                    tag2image[lemmatized_tag].append(len(images) - 1)
                else:
                    tag2image[lemmatized_tag] = [len(images) - 1]
                image_tags[-1].append(tag)

                if counter == TOPK:
                    break
print('Num tags:', len(tag2image))
print('Num clean tags:', len(clean_tags))

Num tags: 2839
Num clean tags: 2981
CPU times: user 3min 15s, sys: 5.99 s, total: 3min 21s
Wall time: 3min 39s


In [11]:
import json

with open('../data/interim/yfcc100m/tag_to_images-0.json', 'w') as fid:
    json.dump({'images': images, 'image_tags': image_tags, 'tag2image': tag2image}, fid)

# Check relevance of top-k tags in YFCC100M

Compare raw top-k tags vs relevant* top-k tags for about 100 random images.

- This generates the data from the webpage `yfcc100m_original_filtered_v2`

* relevant means used in [this project](http://deep-tagging.cs.washington.edu/imagenet_correspondence.html)

~~goal: visualize 100 random images with top-5 tags vs filtered top-5 tags~~

In [3]:
%%time

import csv
filename = 'data/interim/yfcc100m/tag_frequency.csv'
clean_tags = set()
with open(filename, 'r') as fid:
    reader = csv.DictReader(fid, delimiter=',')
    for row in reader:
        clean_tags.add(row['tag'])

from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

import glob
import csv
# Those CSV are too big and we need these two lines
import sys
csv.field_size_limit(sys.maxsize)
import random

SEED = 13029
if SEED is not None:
    random.seed(SEED)
TOPK = 5
IMAGE_ID = 0
USER_ID = 1
USER_TAGS = 8
IMAGE_URL = 14
PHOTO_OR_VIDEO = 22
SUBSET = 180 # None

rows = []
image_tag_mapping = {}
for i in glob.glob('/mnt/ilcompf2d1/data/yfcc100m/yfcc100m_dataset-0*'):
    with open(i, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            
            if int(row[PHOTO_OR_VIDEO]) != 0 or len(row[USER_TAGS]) == 0:
                continue
            if SUBSET is not None:
                if random.random() > SUBSET / 10000000:
                    continue  
            
            counter, counter2 = 0, 0
            image_url = f'{row[IMAGE_URL]}'
            topk_original_tags, topk_filtered_tags = [], []
            for tag in row[USER_TAGS].split(','):
                if counter < TOPK:
                    counter += 1
                    topk_original_tags.append(tag)
                    
                lemmatized_tag = lemmatizer(tag, u'NOUN')[0]
                if lemmatized_tag in clean_tags:
                    counter2 += 1
                    topk_filtered_tags.append(lemmatized_tag)
                    
                if counter2 == TOPK:
                    break
            image_tag_mapping[image_url] = {'original': topk_original_tags,
                                            'filtered': topk_filtered_tags}
print('Num images:', len(image_tag_mapping))

CPU times: user 57.4 s, sys: 1.23 s, total: 58.7 s
Wall time: 58.8 s


In [5]:
import json

with open('data/interim/yfcc100m/top5_tags_100.json', 'w') as fid:
    json.dump(image_tag_mapping, fid)